### Import Section

In [1]:
import sys
import io
import os
import urllib.parse, urllib.request, json
import pandas as pd
import numpy as np
from typing import Iterable, Tuple, List, Set, Dict, Callable, Any, Optional
from pprint import PrettyPrinter
from collections import Counter,defaultdict
from tqdm import tqdm
pprint = PrettyPrinter(indent=4)
print = pprint.pprint

### Constant Section
Important!!! Change the *_PATH according to your configuration

In [2]:
DATA_PATH = os.path.join("c:\\","Users","alexp","Jupyter","NU","github","data","CE_SC_papers.csv")
GROUPS_PATH = os.path.join("c:\\","Users","alexp","Jupyter","NU","github","data","groups.txt")
WIKIFIER_PATH =  os.path.join("c:\\","Users","alexp","Jupyter","NU","github","data","wikifier.pkl")
#Enter your credentials 
USERNAME = ""
KEY = ""

### Reference and Data Section

In [3]:
#### Vocabulary of concepts
voc_conc = Counter()
#### Vocabulary of clazzes
voc_claz = Counter()
#### Generalizations of concepts to clazzes (is-a)
is_a = defaultdict(set)

with open(WIKIFIER_PATH,"rb") as fh:
    df = pd.read_pickle(fh)

with open(GROUPS_PATH,"r") as fh:
    ref_is_a = json.loads(fh.read())

GROUPS = dict()    
for k,v in ref_is_a.items():
    is_a[k].add(v)
    voc_conc[k]+=1
    voc_claz[v]+=1
    GROUPS[v] = ({v},{})
    
STOP_WORDS = {
    "Creative Commons",
    "Copyright"
}

PROBLEM_WORDS = [
    "Steel",
    "Slag",
    "Rice",
    "Water",
    "Limestone",
    "Mineral",
    "Greenhouse gas",
    "Geopolymer",
    "Fossil fuel",
    "Fly ash",
    "Concrete","Composite material",
    "Alkali",
    "Wood"]

SYN_SETS = {
    "Sustainability":"Sustainability",
    "Sustainable development":"Sustainability",
    "Sustainable Development Goals":"Sustainability",
    "Sustainable architecture":"Sustainability",
}



### Routine Section

In [28]:
def chunk_apply(self, source: str, target: str, func: Callable, parts: int, start: int = 0):
    """
    Applies a function to chunks of a DataFrame column and stores the results in another column.

    This method divides a DataFrame into specified parts and applies a function to a
    column (`source`) in chunks, then stores the results in another column (`target`).
    It uses a progress bar via tqdm to display progress.

    Parameters:
    - self: The DataFrame to operate on.
    - source (str): The name of the column from which data is read.
    - target (str): The name of the column where results of the function are stored.
    - func (Callable): The function to apply to each element of the source column.
    - parts (int): The number of parts to divide the DataFrame into for processing.
    - start (int, optional): The starting index from which to begin processing. Default is 0.

    Returns:
    - None: The function modifies the DataFrame in place and does not return anything.
    """
    from tqdm import tqdm
    tqdm.pandas(desc="Chunk apply")
    chunk_size = int(len(self) / parts)
    for chunk_start in range(start, len(self), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(self))
        self.loc[chunk_start:chunk_end, target] = self.loc[chunk_start:chunk_end, source].progress_apply(func)

def call_wikifier(text: str, lang: str = "en", threshold: float = 0.8, retry: int = 0) -> Optional[List[Tuple[str, str]]]:
    """
    Calls the Wikifier service to annotate text with relevant Wikipedia links.
    
    Args:
    text (str): The text to be annotated.
    lang (str, optional): The language of the text. Defaults to "en" (English).
    threshold (float, optional): The page rank square threshold for selecting annotations. Defaults to 0.8.
    retry (int, optional): The retry count for handling request failures. Automatically retries up to 5 times. Defaults to 0.
    
    Returns:
    Optional[List[Tuple[str, str]]]: A list of tuples containing the title and URL of the Wikipedia page if successful, None otherwise.
    
    This function constructs a URL request to the Wikifier service, handling retries and errors. If the number of retries exceeds 5, it returns None. For each successful call, it returns a list of annotated Wikipedia links based on the specified threshold.
    """
    # Prepare the URL.
    if retry > 5:
        return None
    if retry > 0:
        time.sleep(retry)
    try:
        data = urllib.parse.urlencode([
            ("text", text), ("lang", lang),
            ("userKey", "Your_User_Key_Here"),  # Replace with your actual user key.
            ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
            ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
            ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
            ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
            ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
        url = "http://www.wikifier.org/annotate-article"
        # Call the Wikifier and read the response.
        req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
        with urllib.request.urlopen(req, timeout=60) as f:
            response = f.read()
            response = json.loads(response.decode("utf8"))
        # Output the annotations.
        return [(annotation["title"], annotation["url"]) for annotation in response["annotations"]]
    except Exception as error:
        print(f"Error on attempt {retry+1}: {error}")
        return call_wikifier(text, lang, threshold, retry + 1)

def get_concept_info(title: str, lang: str = "en", secLang: str = "ru", retry: int = 0):
    """
    Retrieves concept information from the Wikifier service.

    This function queries the Wikifier "concept-info" API to get detailed information about a specified concept,
    including its labels in different languages. It supports retrying the request in case of failures.

    Parameters:
    title (str): The title of the concept to query.
    lang (str): The primary language for the query (default is English).
    secLang (str): A secondary language for additional labels (default is Russian).
    retry (int): The number of retry attempts made in case of a failure.

    Returns:
    list of str: A list of labels of the concept in the secondary language, or an empty list on failure.

    Raises:
    Exception: Propagates exceptions after exceeding retry limits with a descriptive error message.
    """
    # Prepare the URL.
    if retry > 5:
        raise Exception("Maximum retry attempts exceeded.")
    if retry > 0:
        time.sleep(retry)  # Backoff before retrying

    try:
        base_url = "http://www.wikifier.org/concept-info"
        params = {
            "lang": lang,
            "title": title,
            "secLang": secLang
        }
        url = base_url + "?" + urllib.parse.urlencode(params)

        with urllib.request.urlopen(url) as response:
            data = response.read().decode('utf-8')
            concept_info = json.loads(data)
            return [el["enLabel"] for el in concept_info["wikiDataClasses"]]
    except Exception as e:
        print("Error:", e)
        return get_concept_info(title, lang, secLang, retry + 1)

def get_hyperonyms(term: str, lang: str = 'en') -> list:
    """
    Retrieve hyperonyms (superordinate terms) for a given term using the ConceptNet API.

    Args:
    term (str): The term for which to find hyperonyms.
    lang (str): The language of the term (default is 'en' for English).

    Returns:
    list: A list of hyperonyms for the specified term. Returns None if the API request fails.

    Example:
    >>> hyperonyms = get_hyperonyms('apple')
    >>> print("Hyperonyms of 'apple':", hyperonyms)
    """
    
    # Construct the URL and parameters for the ConceptNet API
    base_url = "http://api.conceptnet.io/query"
    params = {
        'node': f'/c/{lang}/{term}',
        'rel': '/r/IsA',
        'limit': 10  # limit the results to 10; adjust as needed
    }
    
    # Encode parameters and construct full URL
    query_string = urllib.parse.urlencode(params)
    url = f"{base_url}?{query_string}"
    
    try:
        # Make the HTTP GET request to the ConceptNet API
        with urllib.request.urlopen(url) as response:
            # Decode the response and load it into JSON
            response_text = response.read().decode('utf-8')
            data = json.loads(response_text)
            hyperonyms = []
            
            # Extract hyperonyms from the API response
            for edge in data['edges']:
                # Get the start or end node as a hyperonym depending on the direction
                if edge['start']['@id'] == f'/c/{lang}/{term}':
                    hyperonym = edge['end']['label']
                else:
                    hyperonym = edge['start']['label']
                hyperonyms.append(hyperonym)
            
            return hyperonyms
    except urllib.error.URLError as e:
        print("Failed to fetch data:", e)
        return None

def flag_clazz(text: str, features: Tuple[Set[str], Set[str]]) -> bool:
    """
    Determine if a text contains any substrings from the first set in 'features'
    and none from the second set.
    
    Args:
    text (str): The text to check.
    features (Tuple[Set[str], Set[str]]): A tuple containing two sets of substrings,
                                         the first to check for presence and the second for absence.
    
    Returns:
    bool: True if conditions are met, False otherwise.
    """
    return any(subs in text for subs in features[0]) and not any(subs in text for subs in features[1])

def filter_clazz(texts: Iterable[str], groups: Dict[str, Tuple[Set[str], Set[str]]]):
    """
    A generator function that yields tuples of group keys and texts from a collection
    where the texts match the feature criteria of the corresponding group.
    
    Args:
    texts (Iterable[str]): A collection of text strings to be filtered.
    groups (Dict[str, Tuple[Set[str], Set[str]]]): A dictionary of groups, each associated with
                                                   a tuple of sets defining the inclusion and exclusion criteria.
    
    Yields:
    Tuple[str, str]: Tuples where the first element is a group key and the second is a text matching that group's criteria.
    """
    res = []
    for k, v in groups.items():
        for text in texts:
            if flag_clazz(text, v):
                yield (k, text)

def filter_clazz(texts: Iterable[str], groups: Dict[str, Tuple[Set[str], Set[str]]]) -> str:
    """
    Filters texts by applying group criteria from 'groups' and returns the most common group
    label based on the frequency of matches, or a default label if no matches are found.
    
    Args:
    texts (Iterable[str]): A collection of text strings to be filtered.
    groups (Dict[str, Tuple[Set[str], Set[str]]]): A dictionary of groups with their respective features.
    
    Returns:
    str: The most common group label or "Core" if no matches are found.
    """
    labels = [k for text in texts for k, v in groups.items() if flag_clazz(text, v)]
    if labels:
        buff = Counter(labels)
        return buff.most_common()[0][0]
    return "Core"

def filter_clazz(concept: Any, groups: Dict[str, Tuple[Set[str], Set[str]]], default:str = None) -> str:
    """
    Determines the group classification of a concept based on the predefined groups.

    Args:
    concept (Any): The concept to be classified.
    groups (Dict[str, Tuple[Set[str], Set[str]]]): Groups with their respective criteria for classification.

    Returns:
    str: The classification of the concept, if any.
    """
    for group_name, (includes, excludes) in groups.items():
        if any(include in concept for include in includes) and not any(exclude in concept for exclude in excludes):
            return group_name
    return default

def obsolete():
    for i in range(len(df)):
        for j in range(len(df["concepts"].iloc[i])):
            text = df["concepts"].iloc[i][j][0]
            if text not in STOP_WORDS:

                voc_conc[text] += 1
                if df["concepts"].iloc[i][j][1]:
                    for k in range(len(df["concepts"].iloc[i][j][1])):
                        voc_claz[df["concepts"].iloc[i][j][1][k]] += 1 
                        is_a[text].add(df["concepts"].iloc[i][j][1][k])
                else:
                    is_a[text].add("unknown")


    is_a["Value chain"]=set()
    is_a["Value chain"].add("unknown")

    is_a["Sustainable Development Goals"]=set()
    is_a["Sustainable Development Goals"].add("unknown")

    for word in PROBLEM_WORDS:
        for el in ["material","chemical","substance","element","component"]:
            is_a[word].add(el)
#obsolete()

def build_concept_matrix(df: pd.DataFrame, stop_words: Set[str]) -> Counter:
    """
    Builds a co-occurrence matrix for concepts listed in each row of a DataFrame,
    excluding any concepts that are contained within a set of stop words.

    Args:
    df (pd.DataFrame): A DataFrame containing a column 'concepts' where each cell is a list of tuples,
                       each tuple containing a concept and potentially other associated data.
    stop_words (Set[str]): A set of words to exclude from the matrix calculation.

    Returns:
    Counter: A Counter object representing the co-occurrence matrix where the keys are tuples of concepts
             and the values are the counts of their co-occurrences in the DataFrame.
    """
    concept_matrix = Counter()
    for concepts in df['concepts']:
        for i, el1 in enumerate(concepts):
            for j in range(i + 1, len(concepts)):
                el2 = concepts[j]
                if el1[0] not in stop_words and el2[0] not in stop_words:
                    concept_matrix[(el1[0], el2[0])] += 1
                    concept_matrix[(el2[0], el1[0])] += 1
    return concept_matrix

# TODO el1[0], el2[0] => el1, el2 
def build_concept_matrix(concepts_series: pd.Series, stop_words: Set[str], syn_sets: Dict[str,str] = None) -> Counter:
    """
    Builds a co-occurrence matrix for concepts listed in each element of a pandas Series,
    excluding any concepts that are contained within a set of stop words.

    Args:
    concepts_series (pd.Series): A Series where each element is a list of tuples,
                                 each tuple containing a concept and potentially other associated data.
    stop_words (Set[str]): A set of words to exclude from the matrix calculation.

    Returns:
    Counter: A Counter object representing the co-occurrence matrix where the keys are tuples of concepts
             and the values are the counts of their co-occurrences in the Series.
    """
    concept_matrix = Counter()
    for concepts in concepts_series:
        for i, el1 in enumerate(concepts):
            for j in range(i + 1, len(concepts)):
                el2 = concepts[j]
                if el1[0] not in stop_words and el2[0] not in stop_words:
                    if not syn_sets:
                        concept_matrix[(el1[0], el2[0])] += 1
                        concept_matrix[(el2[0], el1[0])] += 1
                    else:
                        key1 = syn_sets[el1[0]] if el1[0] in syn_sets else el1[0]
                        key2 = syn_sets[el2[0]] if el2[0] in syn_sets else el2[0]
                        concept_matrix[(key1,key2)] += 1
                        concept_matrix[(key2,key1)] += 1
    return concept_matrix

def filter_by_percentile(concept_matrix: Counter, percentile: float = 90) -> List[Tuple[Tuple[str, str], int]]:
    """
    Filters concept pairs from a co-occurrence matrix that meet or exceed the specified percentile.

    Args:
    concept_matrix (Counter): The co-occurrence matrix of concepts.
    percentile (float): The percentile to use as a cutoff for filtering.

    Returns:
    List[Tuple[Tuple[str, str], int]]: A list of concept pairs with their frequencies that meet or exceed the cutoff.
    """
    freqs = np.array([el[1] for el in concept_matrix.most_common()])
    cutoff = np.percentile(freqs, percentile)
    print(f'Cutoff frequency: {cutoff}')
    return [(pair, freq) for pair, freq in concept_matrix.items() if freq >= cutoff]

def filter_by_degree(concept_matrix: Counter, cutoff: float = 50) -> List[Tuple[Tuple[str, str], int]]:
    """
    Filters concept pairs from co-occurence matrix that have particular degree.
    
    Args:
    concept_matrix (Counter): The co-occurence matrix of concepts.
    percentile (float): The percentile to use as a cutodd for filtering.
    
    Returns:
    List[Tuple[Tuple[str,str],int]: A list of concept pairs with their freqs that have the concrete degree.
    
    """
    degree = Counter()
    for el in concept_matrix.items():
        degree[el[0][0]]+=1
    # freqs = np.array([el[1] for el in degree.most_common()])
    # cutoff = np.percentile(freqs, percentile)
    concepts = [el[0] for el in degree.most_common() if el[1]>cutoff]
    print(f'Cutoff degree: {cutoff}')
    return [(pair, freq) for pair, freq in concept_matrix.items() if pair[0] in concepts or pair[1] in concepts]

def counter_to_dataframe(filtered_pairs: List[Tuple[Tuple[str, str], int]], is_a: Dict[str, Any], groups: Dict[str, Tuple[Set[str], Set[str]]]) -> pd.DataFrame:
    """
    Transforms a list of filtered concept pairs into a DataFrame with additional categorization based on provided groups.

    Args:
    filtered_pairs (List[Tuple[Tuple[str, str], int]]): A list of concept pairs and their frequencies.
    is_a (Dict[str, Any]): A dictionary mapping concepts to categories or other properties.
    groups (Dict[str, Tuple[Set[str], Set[str]]]): A dictionary defining groups and their inclusion/exclusion criteria for categorization.

    Returns:
    pd.DataFrame: A DataFrame with categorized concept pairs and their frequencies.
    """
    buff = []
    for (el1, el2), freq in filtered_pairs:
        clazz1 = filter_clazz(is_a[el1], groups)
        clazz2 = filter_clazz(is_a[el2], groups)
        if clazz1 and clazz2:
            buff.append({
                "Section(Row)": clazz1,
                "Section(Column)": clazz2,
                "Name(Row)": el1,
                "Name(Column)": el2,
                "Frequency": freq
            })
    return pd.DataFrame(buff)

def create_matrix_from_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Processes a DataFrame to sort by specified criteria, extract unique indices, 
    and populate a new DataFrame with multi-level indices and columns based on frequencies.

    Args:
    df (pd.DataFrame): The input DataFrame that contains 'Section' and 'Name' categories with 'Frequency' data.

    Returns:
    pd.DataFrame: A new DataFrame where rows and columns are multi-level indices representing unique 'Section' and 'Name' combinations, and the cell values are 'Frequency' from the original DataFrame.
    """
    # Sort by row identifiers and calculate unique row indices
    # df = df.sort_values(by=['Section(Row)', 'Name(Row)', 'Frequency'], ascending=False)
    df = df.sort_values(by=['Section(Row)', 'Frequency'], ascending=False)

    unique_rows = df[['Section(Row)', 'Name(Row)']].drop_duplicates()
    indices = pd.MultiIndex.from_tuples(unique_rows.values.tolist())
    print(f"Number of unique rows: {len(indices)}")

    # Sort by column identifiers and calculate unique column indices
    #df = df.sort_values(by=['Section(Column)', 'Name(Column)', 'Frequency'], ascending=False)
    df = df.sort_values(by=['Section(Column)', 'Frequency'], ascending=False)
    unique_columns = df[['Section(Column)', 'Name(Column)']].drop_duplicates()
    columns = pd.MultiIndex.from_tuples(unique_columns.values.tolist())
    print(f"Number of unique columns: {len(columns)}")

    # Initialize an empty data array to create the DataFrame
    data = np.zeros((len(indices), len(columns)))

    # Create the DataFrame with specified indices and columns
    result = pd.DataFrame(data, index=indices, columns=columns)

    # Populate the DataFrame using the frequencies from the original DataFrame
    for i in tqdm(range(len(df)), desc="Populating matrix"):
        result.loc[
            (df.iloc[i]['Section(Row)'], df.iloc[i]['Name(Row)']),
            (df.iloc[i]['Section(Column)'], df.iloc[i]['Name(Column)'])
        ] = df.iloc[i]['Frequency']

    return result

def top_concepts(df:pd.DataFrame, core_concept:str, ascending=False) -> pd.DataFrame:
    # Assume `df` is your DataFrame containing Numerical coefficients
    # and `CORE_CONCEPTS` is a list of concepts you are interested in.

    # Extract the Series for the first concept in CORE_CONCEPTS
    series = df[core_concept]

    # Ensure we have a Series
    if isinstance(series, pd.DataFrame):
        series = series.iloc[:, 0]

    # Sort the Series in descending order
    sorted_series = series.sort_values(ascending=ascending)

    # Get the top 10 correlated concepts, excluding the first one which is the concept itself
    top_series = sorted_series

    # Display the top 10 correlated concepts and their correlation values
    return pd.DataFrame([{"Concept":concept[1], "Value":f"{value:.2f}"} for concept, value in top_series.items()])
        

def create_section_matrix_from_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(by=['Section(Row)', 'Frequency'], ascending=False)

    unique_rows = df[['Section(Row)']].drop_duplicates()
    indices = pd.MultiIndex.from_tuples(unique_rows.values.tolist())
    print(f"Number of unique rows: {len(indices)}")

    # Sort by column identifiers and calculate unique column indices
    #df = df.sort_values(by=['Section(Column)', 'Name(Column)', 'Frequency'], ascending=False)
    df = df.sort_values(by=['Section(Column)', 'Frequency'], ascending=False)
    unique_columns = df[['Section(Column)']].drop_duplicates()
    columns = pd.MultiIndex.from_tuples(unique_columns.values.tolist())
    print(f"Number of unique columns: {len(columns)}")
    # Initialize an empty data array to create the DataFrame
    data = np.zeros((len(indices), len(columns)))
    # Create the DataFrame with specified indices and columns
    result = pd.DataFrame(data, index=indices, columns=columns)
    for i in tqdm(range(len(df)), desc="Populating matrix"):
        result.loc[
            (df.iloc[i]['Section(Row)']),
            (df.iloc[i]['Section(Column)'])
        ] += df.iloc[i]['Frequency']    
    return result    

pd.DataFrame.chunk_apply = chunk_apply

#print([el for el in voc_claz.items()  if flag_clazz(el[0],GROUPS["Technology"]) ])
#print([el for el in voc_claz.items()  if flag_clazz(el[0],GROUPS["Material"]) ])
#print([el for el in voc_claz.items() if flag_clazz(el[0],GROUPS["Method"]) ])



# Example usage
#term = "matrix"
#hyperonyms = get_hyperonyms(term)
#print(("Hyperonyms of", term, ":", hyperonyms))
#for term in PROBLEM_WORDS:
#    print(get_hyperonyms(term))    
#pd.DataFrame.chunk_apply(func = None,parts = 10,start = 0)

#### Get Wiki concepts from Texts

In [ ]:
df.chunk_apply("abstract","concepts",func = call_wikifier,parts = 10,start = 0)

#### Get Wiki classes (is-a)

In [ ]:
df.chunk_apply("concepts","concepts",func = lambda concepts: [(el[0],get_concept_info(el[0])) for el in concepts],parts = 10,start = 8)


#### Create Frequency Vocabular of Concepts, Clazzz and Is-A relation

In [5]:
bufff = build_concept_matrix(df["concepts"],STOP_WORDS,SYN_SETS)

In [6]:
bufff.most_common()[:10]

[(('Construction', 'Sustainability'), 144),
 (('Sustainability', 'Construction'), 144),
 (('Technology', 'Sustainability'), 91),
 (('Sustainability', 'Technology'), 91),
 (('Sustainability', 'Circular economy'), 75),
 (('Circular economy', 'Sustainability'), 75),
 (('Sustainability', 'Energy'), 67),
 (('Energy', 'Sustainability'), 67),
 (('Sustainability', 'Sustainability'), 66),
 (('Construction', 'Circular economy'), 64)]

In [7]:
fltr_list = filter_by_percentile(bufff)
print(len(fltr_list))

'Cutoff frequency: 2.0'
9871


In [8]:
fltr_list = filter_by_degree(bufff, cutoff = 50)
print(len(fltr_list))

'Cutoff degree: 50'
40749


In [9]:

lin_df = counter_to_dataframe(fltr_list,is_a,GROUPS)
print(len(lin_df))

3145


In [10]:

result = create_matrix_from_dataframe(lin_df)

'Number of unique rows: 63'
'Number of unique columns: 63'


Populating matrix: 100%|█████████████████████████████████████████████████████████| 3145/3145 [00:02<00:00, 1384.75it/s]


In [11]:
result

Waste Management  \
                                                       Recycling   
Waste Management  Recycling                                  0.0   
                  Waste management                          21.0   
                  Waste                                     12.0   
                  Landfill                                   8.0   
                  Construction waste                         9.0   
...                                                          ...   
Built Environment Infrastructure                             3.0   
                  Green building                             5.0   
                  Architecture                               0.0   
                  Building information modeling              1.0   
                  Urbanization                               2.0   

                                                                        \
                                                Waste management Waste   
Waste Management  Recycling                                 21.0  12.0   
                  Waste management                           0.0   9.0   
                  Waste                                      9.0   0.0   
                  Landfill                                  11.0   8.0   
                  Construction waste                        11.0   5.0   
...                                                          ...   ...   
Built Environment Infrastructure                             3.0   1.0   
                  Green building                             3.0   2.0   
                  Architecture                               0.0   0.0   
                  Building information modeling              1.0   0.0   
                  Urbanization                               1.0   2.0   

                                                                             \
                                                Landfill Construction waste   
Waste Management  Recycling                          8.0                9.0   
                  Waste management                  11.0               11.0   
                  Waste                              8.0                5.0   
                  Landfill                           0.0                2.0   
                  Construction waste                 2.0                0.0   
...                                                  ...                ...   
Built Environment Infrastructure                     0.0                2.0   
                  Green building                     3.0                2.0   
                  Architecture                       0.0                0.0   
                  Building information modeling      0.0                1.0   
                  Urbanization                       0.0                0.0   

                                                Sustainability  \
                                                Sustainability   
Waste Management  Recycling                               44.0   
                  Waste management                        34.0   
                  Waste                                   28.0   
                  Landfill                                15.0   
                  Construction waste                      14.0   
...                                                        ...   
Built Environment Infrastructure                          25.0   
                  Green building                          24.0   
                  Architecture                            17.0   
                  Building information modeling           13.0   
                  Urbanization                            12.0   

                                                Science and Research           \
                                                   Scientific method Research   
Waste Management  Recycling                                     12.0      6.0   
                  Waste management                              10.0  

In [ ]:
result.to_excel('concept_matrix_top.xlsx', engine='openpyxl')

In [12]:
CORE_CONCEPTS = ["Circular economy","Sustainability"]
df = lin_df.sort_values(by=['Section(Row)', 'Frequency'], ascending=False)
unique_rows = df[['Section(Row)']].drop_duplicates()

In [13]:
pearson_corr = result.corr(method='pearson')

In [14]:
pearson_corr

Waste Management  \
                                                       Recycling   
Waste Management  Recycling                             1.000000   
                  Waste management                      0.835734   
                  Waste                                 0.886405   
                  Landfill                              0.877982   
                  Construction waste                    0.832457   
...                                                          ...   
Built Environment Infrastructure                        0.802644   
                  Green building                        0.691586   
                  Architecture                          0.708604   
                  Building information modeling         0.632086   
                  Urbanization                          0.656201   

                                                                            \
                                                Waste management     Waste   
Waste Management  Recycling                             0.835734  0.886405   
                  Waste management                      1.000000  0.841768   
                  Waste                                 0.841768  1.000000   
                  Landfill                              0.833745  0.820570   
                  Construction waste                    0.842007  0.762876   
...                                                          ...       ...   
Built Environment Infrastructure                        0.753893  0.750528   
                  Green building                        0.675705  0.679146   
                  Architecture                          0.679766  0.637555   
                  Building information modeling         0.638101  0.532815   
                  Urbanization                          0.591143  0.648620   

                                                                              \
                                                 Landfill Construction waste   
Waste Management  Recycling                      0.877982           0.832457   
                  Waste management               0.833745           0.842007   
                  Waste                          0.820570           0.762876   
                  Landfill                       1.000000           0.887873   
                  Construction waste             0.887873           1.000000   
...                                                   ...                ...   
Built Environment Infrastructure                 0.704894           0.737303   
                  Green building                 0.580117           0.602331   
                  Architecture                   0.596270           0.633675   
                  Building information modeling  0.559750           0.643005   
                  Urbanization                   0.550797           0.552493   

                                                Sustainability  \
                                                Sustainability   
Waste Management  Recycling                           0.760182   
                  Waste management                    0.772666   
                  Waste                               0.712459   
                  Landfill                            0.760387   
                  Construction waste                  0.780042   
...                                                        ...   
Built Environment Infrastructure                      0.728704   
                  Green building                      0.643475   
                  Architecture                        0.692821   
                  Building information modeling       0.677300   
                  Urbanization                        0.606973   

                                                Science and Research  \
                                                   Scientific method   
Waste Management  Recycling                                 0.781847   
                  Waste manageme

In [17]:
df_cfreq = pd.concat([top_concepts(result,CORE_CONCEPTS[0]).iloc[0:10],top_concepts(result,CORE_CONCEPTS[1]).iloc[0:10]], keys = CORE_CONCEPTS,axis=1)
df_cfreq = df_cfreq.style.set_caption('Top 10 Most Frequent Concept Pairs with "Circular Economy" and "Sustainable Development"')


In [18]:
df_cfreq

In [19]:
df_cpearson = pd.concat([top_concepts(pearson_corr,CORE_CONCEPTS[0]).iloc[0:11],top_concepts(pearson_corr,CORE_CONCEPTS[1]).iloc[0:11]], keys = CORE_CONCEPTS, axis=1)
df_cpearson = df_cpearson.style.set_caption('Top 10 Concept Pairs Most Strongly Correlated with "Circular Economy" and "Sustainable Development" Using Pearson Correlation')


In [20]:
df_cpearson_bottom = pd.concat([top_concepts(pearson_corr,CORE_CONCEPTS[0]).iloc[-11:],top_concepts(pearson_corr,CORE_CONCEPTS[1]).iloc[-11:]], keys = CORE_CONCEPTS, axis=1)
df_cpearson_bottom

Circular economy              Sustainability      
                     Concept Value               Concept Value
52                    Cement  0.65          Urbanization  0.61
53                   Machine  0.64                  Clay  0.61
54         Materials science  0.63               Machine  0.60
55                 Pollution  0.63                System  0.59
56  Research and development  0.62        Climate change  0.59
57                    System  0.62          Biodiversity  0.58
58      Compressive strength  0.61     Materials science  0.58
59               Information  0.61    Composite material  0.57
60               Open access  0.58      Carbon footprint  0.57
61          Carbon footprint  0.56  Efficient energy use  0.54
62      Efficient energy use  0.56          Construction  0.53

In [21]:
df_cpearson

In [ ]:
pearson_corr.to_excel('pearson_corr.xlsx', engine='openpyxl')

In [22]:
spearman_corr = result.corr(method='spearman')

In [23]:
df_cspearman = pd.concat([top_concepts(spearman_corr,CORE_CONCEPTS[0]).iloc[0:11],top_concepts(spearman_corr,CORE_CONCEPTS[1]).iloc[0:11]], axis=1)
df_cspearman = df_cspearman.style.set_caption('Top 10 Concept Pairs Most Strongly Correlated with "Circular Economy" and "Sustainable Development" Using Spearman Correlation')


In [24]:
df_cspearman_bottom = pd.concat([top_concepts(spearman_corr,CORE_CONCEPTS[0]).iloc[-11:],top_concepts(spearman_corr,CORE_CONCEPTS[1]).iloc[-11:]], axis=1)

In [25]:
df_cspearman_bottom

,Concept,Value,Concept,Value
52,Information,0.32,Efficient energy use,0.47
53,Architecture,0.31,Decision-making,0.46
54,Environmentally friendly,0.31,Carbon footprint,0.46
55,Carbon,0.31,Literature,0.46
56,Clay,0.30,Machine,0.46
57,Cement,0.28,Cement,0.43
58,Efficient energy use,0.28,Economics,0.42
59,Carbon footprint,0.27,Bibliometrics,0.41
60,Machine,0.25,Clay,0.40
61,Design,0.25,Composite material,0.40


In [26]:
spearman_corr.loc[CORE_CONCEPTS[0]]

Waste Management                                       \
                        Recycling Waste management     Waste  Landfill   
Circular economy          0.60624         0.614358  0.458315  0.571665   

                                    Sustainability Science and Research  \
                 Construction waste Sustainability    Scientific method   
Circular economy           0.655209        0.62939             0.526191   

                                                 ...       Economics  \
                  Research Literature   Science  ... Economic growth   
Circular economy  0.485183    0.39022  0.430978  ...        0.520105   

                 Circular economy Built Environment              \
                 Circular economy      Construction Engineering   
Circular economy              1.0          0.638149    0.522988   

                                                                               \
                 Built environment Infrastructure Green building Architecture   
Circular economy          0.431199       0.532068       0.552324     0.313426   

                                                             
                 Building information modeling Urbanization  
Circular economy                      0.424014     0.390608  

[1 rows x 63 columns]

In [27]:
spearman_corr.loc[CORE_CONCEPTS[1]]

Waste Management                                       \
                      Recycling Waste management     Waste  Landfill   
Sustainability         0.657828         0.569058  0.498088  0.557827   

                                  Sustainability Science and Research  \
               Construction waste Sustainability    Scientific method   
Sustainability           0.691614            1.0             0.710458   

                                               ...       Economics  \
                Research Literature   Science  ... Economic growth   
Sustainability  0.580571   0.456945  0.564495  ...        0.648233   

               Circular economy Built Environment              \
               Circular economy      Construction Engineering   
Sustainability          0.62939           0.77289     0.66407   

                                                                             \
               Built environment Infrastructure Green building Architecture   
Sustainability          0.637615       0.683212        0.60027     0.529909   

                                                           
               Building information modeling Urbanization  
Sustainability                      0.470295     0.616807  

[1 rows x 63 columns]

In [29]:
section_matrix = create_section_matrix_from_dataframe(lin_df)

'Number of unique rows: 9'
'Number of unique columns: 9'


Populating matrix: 100%|██████████████████████████████████████████████████████████| 3145/3145 [00:06<00:00, 521.47it/s]


In [30]:
COLUMNS = [
    'Sustainability',
    'Circular economy',
    'Built Environment',
    'Economics',
    'Environment',
    'General Technology',
    'Materials',
    'Science and Research',
    'Waste Management']

section_matrix = section_matrix[COLUMNS].loc[COLUMNS]


for el in ['Sustainability', 'Circular economy', 'Built Environment', 'Economics', 'Environment', 'General Technology', 'Materials', 'Science and Research', 'Waste Management']:
    section_matrix.loc[el,el] = 0

for col in COLUMNS:
    total = section_matrix[col].sum()[0]
    #print((col, total))
    section_matrix[col] = section_matrix[col].apply(lambda x : round(x/total,2))

In [31]:

for col in COLUMNS:
    arr = np.array([section_matrix[col].iloc[ind][0] for ind in range(len(section_matrix[col]))])
    section_matrix[col] = arr/arr.sum()

In [32]:
for col in COLUMNS:
    print(section_matrix[col].sum())

Sustainability    1.0
dtype: float64
Circular economy    1.0
dtype: float64
Built Environment    1.0
dtype: float64
Economics    1.0
dtype: float64
Environment    1.0
dtype: float64
General Technology    1.0
dtype: float64
Materials    1.0
dtype: float64
Science and Research    1.0
dtype: float64
Waste Management    1.0
dtype: float64


In [33]:
for el in ['Sustainability', 'Circular economy', 'Built Environment', 'Economics', 'Environment', 'General Technology', 'Materials', 'Science and Research', 'Waste Management']:
    section_matrix.loc[el,el] = "-"
section_matrix

,Sustainability,Circular economy,Built Environment,Economics,Environment,General Technology,Materials,Science and Research,Waste Management
Sustainability,-,0.09,0.131313,0.12,0.13,0.14,0.141414,0.12,0.1
Circular economy,0.039604,-,0.050505,0.07,0.06,0.05,0.050505,0.06,0.08
Built Environment,0.168317,0.15,-,0.15,0.14,0.19,0.151515,0.18,0.13
Economics,0.049505,0.06,0.050505,-,0.04,0.05,0.020202,0.07,0.04
Environment,0.148515,0.15,0.131313,0.13,-,0.16,0.191919,0.13,0.15
General Technology,0.247525,0.18,0.272727,0.21,0.25,-,0.242424,0.3,0.22
Materials,0.138614,0.1,0.121212,0.05,0.17,0.14,-,0.07,0.18
Science and Research,0.128713,0.14,0.161616,0.2,0.12,0.18,0.070707,-,0.1
Waste Management,0.079208,0.13,0.080808,0.07,0.09,0.09,0.131313,0.07,-
